## Interval Scheduling Algorithm

In [1]:
intervals = [(0, 6), (7, 8), (0, 1), (2, 3), (4, 5), (6, 9), (0, 2), (3, 4), (5, 7)]

finishing_times = sorted(intervals, key=lambda x: x[1]) # O(nlogn)

accepted = [finishing_times[0]]
for (s_i, f_i) in finishing_times[1:]: # O(n)
    if s_i > accepted[-1][1]: # >= in the book
        accepted.append((s_i, f_i))
        
print(accepted)    

[(0, 1), (2, 3), (4, 5), (7, 8)]


## Interval Partitioning Problem (Interval Coloring Problem)

In [2]:
intervals = [
    (4,9), (11,13),
    (1,3), (4,6), (8,10),
    (1,6), (10,13),
    (1,3), (8,10), (11,13)
]

expected = [
    [(1,3), (4,6), (8,10), (11,13)],
    [(1,6), (8,10), (11,13)],
    [(1,3), (4,9), (10,13)],
]

def interval_partitioning(list_of_intervals):
    s_intervals = sorted(intervals, key=lambda x: x[0])
    labels = 0
    intervals_by_label = []
    for interval in s_intervals:
        scheduled = False
        for label, l_intervals in enumerate(intervals_by_label):
            last_interval = l_intervals[-1]
            if interval[0] >= last_interval[1]:
                intervals_by_label[label].append(interval)
                scheduled = True
                break
        if not scheduled:
            labels += 1
            intervals_by_label.append([interval])
    return intervals_by_label

def validate_intervals(result, expected):
    expected = sorted(expected)
    passed = True
    sorted_answer = sorted(result)
    for i,j in zip(sorted_answer, expected):
        for k, l in zip(i, j):
            if not all(x == y for x, y in zip(k, l)):
                print(f"ERROR {k} != {l}")
                passed = False
                break

    if passed:
        print("Test passed")

assorted_intervals = interval_partitioning(intervals)
validate_intervals(assorted_intervals, expected)

Test passed


## Scheduling to minimize lateness

In [3]:
# tuples of si (start time), fi (finish time), and di (deadline)
jobs = [(1, 3, 5), (1, 4, 7),(1, 2, 3)]

s_jobs = sorted(jobs, key=lambda x: x[2])
new_schedule = [s_jobs[0]]
for (s_j, f_j, d_j) in s_jobs[1:]:
    t_j = f_j - s_j
    (s_i, f_i, d_i) = new_schedule[-1]
    new_s = f_i
    new_f = new_s + t_j
    new_schedule.append((new_s, new_f, d_j))
    
print(new_schedule)

[(1, 2, 3), (2, 4, 5), (4, 7, 7)]


# Graphs

## Djikstra's Algorithm

In [4]:
import math
from heapq import heappush, heappop

class PriorityQueue:
    
    def __init__(self):
        self.arr = []
        
    def add(self, value):
        heappush(self.arr, value)
        
    def pop(self):
        return heappop(self.arr)
    
    def is_empty(self):
        return len(self.arr) == 0

class GraphEdge(object):
    def __init__(self, node, distance):
        self.node = node
        self.distance = distance
        
class GraphNode(object):
    def __init__(self, val):
        self.value = val
        self.edges = []

    def add_child(self, node, distance):
        self.edges.append(GraphEdge(node, distance))

    def remove_child(self, del_node):
        if del_node in self.edges:
            self.edges.remove(del_node)

class Graph(object):
    def __init__(self, node_list):
        self.nodes = node_list

    def add_edge(self, node1, node2, distance):
        if node1 in self.nodes and node2 in self.nodes:
            node1.add_child(node2, distance)
            node2.add_child(node1, distance)

    def remove_edge(self, node1, node2):
        if node1 in self.nodes and node2 in self.nodes:
            node1.remove_child(node2)
            node2.remove_child(node1)
            

In [65]:
def djikstra(start_node, end_node):
    visited = []
    q = PriorityQueue()
    #     (s, d, node)
    q.add((0, 0, start_node.value, start_node))
    
    visited_count = 0
    while not q.is_empty():
        # value is unnecessary and it's done to deal with equality
        distance, count, value, node = q.pop()
        
        if value not in visited:
            visited.append(value)
        
        if node == end_node:
            print(visited, end_node.value)
            return distance
        
        for edge in node.edges:
            if edge.node.value not in visited:
                q.add((distance + edge.distance, visited_count + count, edge.node.value, edge.node))

def find_shortest_distance(start_node, end_node):
    return 'Shortest Distance from {} to {} is {}'.format(start_node.value, end_node.value, djikstra(start_node, end_node))

node_s = GraphNode('s')
node_u = GraphNode('u')
node_v = GraphNode('v')
node_y = GraphNode('y')
node_x = GraphNode('x')
node_z = GraphNode('z')

graph = Graph([node_s, node_u, node_v, node_y, node_x, node_z])
graph.add_edge(node_s, node_u, 1)
graph.add_edge(node_s, node_x, 4)
graph.add_edge(node_s, node_v, 2)
graph.add_edge(node_u, node_x, 1)
graph.add_edge(node_v, node_x, 2)
graph.add_edge(node_u, node_y, 3)
graph.add_edge(node_x, node_y, 1)
graph.add_edge(node_x, node_z, 2)
graph.add_edge(node_v, node_z, 3)

print(find_shortest_distance(node_s, node_x))
print(find_shortest_distance(node_s, node_z))
print(find_shortest_distance(node_s, node_y))

['s', 'u', 'v', 'x'] x
Shortest Distance from s to x is 2
['s', 'u', 'v', 'x', 'y', 'z'] z
Shortest Distance from s to z is 4
['s', 'u', 'v', 'x', 'y'] y
Shortest Distance from s to y is 3


## Minimun Spanning Tree

Given the following example graph, we solve the MST using Kruskal and Prim's algorithm.


![example_graph](./GraphExampleForGreedyAlgorithms.png)

In [54]:
node_a = GraphNode('a')
node_b = GraphNode('b')
node_c = GraphNode('c')
node_d = GraphNode('d')
node_e = GraphNode('e')
node_f = GraphNode('f')
node_g = GraphNode('g')

graph = Graph([node_a, node_b, node_c, node_d, node_e, node_f, node_g])
graph.add_edge(node_a, node_b, 2**2)
graph.add_edge(node_a, node_d, 3**2)
graph.add_edge(node_a, node_c, 3**2)
graph.add_edge(node_b, node_c, 4**2)
graph.add_edge(node_b, node_e, 3**2)
graph.add_edge(node_d, node_f, 7**2)
graph.add_edge(node_d, node_c, 5**2)
graph.add_edge(node_c, node_e, 1**2)
graph.add_edge(node_f, node_g, 9**2)
graph.add_edge(node_f, node_e, 8**2)

## Kruskal Algorithm

In [55]:
from collections import deque

class Queue:
    
    def __init__(self):
        self.arr = deque()
        
    def push(self, value):
        self.arr.appendleft(value)
        
    def pop(self):
        return self.arr.pop()
    
    def is_empty(self):
        return len(self.arr) == 0

class DisjointSet: 
    def __init__(self, n): 
        self.rank = [1] * n 
        self.parent = [i for i in range(n)] 
  
    def find(self, x): 
        if (self.parent[x] != x): 
            self.parent[x] = self.find(self.parent[x]) 
        return self.parent[x] 
  
    def union(self, x, y): 
        xset = self.find(x) 
        yset = self.find(y) 
  
        if xset == yset: 
            return
  
        if self.rank[xset] < self.rank[yset]: 
            self.parent[xset] = yset 
        elif self.rank[xset] > self.rank[yset]: 
            self.parent[yset] = xset 
        else: 
            self.parent[yset] = xset 
            self.rank[xset] = self.rank[xset] + 1

In [56]:
def bfs_sort_edges(node):
    q = Queue()
    q.push(node)
    visited = []
    sorted_edges = PriorityQueue()
    count = 0
    while not q.is_empty():
        el = q.pop()
        if el.value not in visited:
            visited.append(el.value)
                
        for edge in el.edges:
            if edge.node.value not in visited:
                sorted_edges.add((edge.distance, count, el, edge.node))
                count += 1
                q.push(edge.node)
    return sorted_edges, visited

"""
1. Sort all the edges in non-decreasing order of their weight.
2. Pick the smallest edge. Check if it forms a cycle with the spanning-tree formed so far. 
    If the cycle is not formed, include this edge. Else, discard it.
3. Repeat step#2 until there are (V-1) edges in the spanning tree.
"""
def kruskal(root_node):
    sorted_edges, nodes = bfs_sort_edges(root_node)

    inverse_mapping = { node_value: idx for idx, node_value in enumerate(nodes) }
    disjoint_set = DisjointSet(len(nodes))

    MST = []
    while not sorted_edges.is_empty():
        (distance, _, from_node, to_node) = sorted_edges.pop()
        fr_val = from_node.value
        to_val = to_node.value
        fr = inverse_mapping[fr_val]
        to = inverse_mapping[to_val]
        if not MST:
            MST.append((distance, fr_val, to_val))
            disjoint_set.union(fr, to)
        else:
            if disjoint_set.find(fr) != disjoint_set.find(to):
                MST.append((distance, fr_val, to_val))
                disjoint_set.union(fr, to)

    return MST

print(kruskal(node_a))

[(1, 'c', 'e'), (4, 'a', 'b'), (9, 'a', 'd'), (9, 'a', 'c'), (49, 'd', 'f'), (81, 'f', 'g')]


## Prim's Algorithm

In [ ]:
visited = [node_a.value]
PriorityQueue()

